<a href="https://colab.research.google.com/github/panashematsaudza/PySpark-Customer-Churn-Prediction/blob/master/Customer_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('customer churn').getOrCreate()

In [0]:
data = spark.sql('SELECT * FROM  customer_churn_csv')

In [0]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [0]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [0]:
data.columns

Out[9]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
data.head()

Out[10]: Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol = 'features')

In [0]:
output = assembler.transform(data)

In [0]:
final_data = output.select('features','churn')

In [0]:
train_data ,test_data = final_data.randomSplit([.7,.3])

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
lr_churn = LogisticRegression(labelCol = 'churn')

In [0]:
model = lr_churn.fit(train_data)

In [0]:
train_sum = model.summary

In [0]:
train_sum.predictions.describe().show()

+-------+-------------------+-------------------+
summary| churn| prediction|
+-------+-------------------+-------------------+
 count| 637| 637|
 mean| 0.1554160125588697|0.11145996860282574|
 stddev|0.36258552397472393| 0.3149481893829468|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
pred_and_labels =  model.evaluate(test_data)

In [0]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[28.0,11128.95,1....| 0|[4.55974734124665...|[0.98964367327117...| 0.0|
[30.0,7960.64,1.0...| 1|[3.48545945831551...|[0.97027120320734...| 0.0|
[30.0,10183.98,1....| 0|[3.19183810534688...|[0.96052597276134...| 0.0|
[30.0,11575.37,1....| 1|[4.36024193137955...|[0.98738585293867...| 0.0|
[31.0,5387.75,0.0...| 0|[2.29629414278024...|[0.90856965608781...| 0.0|
[31.0,10182.6,1.0...| 0|[5.17965666445765...|[0.99440157972408...| 0.0|
[31.0,11297.57,1....| 1|[1.25527307235422...|[0.77821132037351...| 0.0|
[32.0,9036.27,0.0...| 0|[-0.3017936503923...|[0.42511906909793...| 1.0|
[32.0,9885.12,1.0...| 1|[1.92031770798214...|[0.87217385805817...| 0.0|
[32.0,12142.99,0....| 0|[5.87613562429045...|[0.99720224313410...| 0.0|
[33.0,8556.73,0.0...| 0|[3.85767427527875...|[0.97931965310802...| 0.0|
[33.0,12249.96,0....| 0|[5.90326105929416...|[0.99727690959154...| 0.0|
[33.0,12638.51,1....| 0|[4.29459454595513...|[0.98654150040043...| 0.0|
[34.0,6131.92,0.0...| 0|[3.77059270380240...|[0.97748041110350...| 0.0|
[34.0,6461.86,1.0...| 0|[4.37209754279198...|[0.98753266490179...| 0.0|
[34.0,7818.13,0.0...| 0|[3.77077162856217...|[0.97748434934062...| 0.0|
[34.0,8772.26,0.0...| 0|[2.79910346548904...|[0.94262735783458...| 0.0|
[34.0,9228.84,1.0...| 1|[-0.7934859928526...|[0.31142064969878...| 1.0|
[34.0,9270.9,0.0,...| 0|[4.94019752167386...|[0.99289761932160...| 0.0|
[34.0,9401.99,1.0...| 0|[0.92955170765885...|[0.71698432784588...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol ='prediction',labelCol='churn')

In [0]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [0]:
auc

Out[31]: 0.7630873103958564

In [0]:
final_lr_model = lr_churn.fit(final_data)

In [0]:
test_customers = spark.sql('SELECT * FROM new_customers')

In [0]:
test_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [0]:
test_new_customers = assembler.transform(test_customers)

In [0]:
final_results =  final_lr_model.transform(test_new_customers)

In [0]:
final_results.select('Company','probability','prediction').show()

+----------------+--------------------+----------+
 Company| probability|prediction|
+----------------+--------------------+----------+
 King Ltd|[0.90218018099700...| 0.0|
 Cannon-Benson|[0.00198380445828...| 1.0|
Barron-Robertson|[0.02255110110417...| 1.0|
 Sexton-Golden|[0.00608622642083...| 1.0|
 Wood LLC|[0.75115067517455...| 0.0|
 Parks-Robbins|[0.15582819767621...| 1.0|
+----------------+--------------------+----------+